### Machine Learning Models for Training and Testing in the Identification of MDM2-P53 Inhibitors ###

This Jupyter notebook helps users develop and evaluate machine-learning models for structure-based virtual screening, specifically targeting MDM2 at the p53 binding site. The models are classification-based and can be trained with or without hyperparameter tuning and cross-validation. Users can select either PLEC or GRID features for their analysis.

This methodology and code are based on the Nature Protocols paper: Tran-Nguyen, V. K., Junaid, M., Simeon, S., & Ballester, P. J. (2023). A practical guide to machine-learning scoring for structure-based virtual screening. We have made some changes to hyperparameter optimization and added cross-validation for each model.

Before running the notebook, we suggest configuring the protocol-env environment. This can be done using the protocol-env.yml file available in the https://github.com/vktrannguyen/MLSF-protocol.

# Import python dependencies #

In [ ]:
import os
import numpy as np
import pandas as pd
import oddt
import oddt.pandas as opd
from oddt.pandas import ChemDataFrame
from oddt.fingerprints import PLEC
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
from scipy import stats
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, precision_recall_curve, accuracy_score, auc
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn.utils import parallel_backend
from sklearn.model_selection import KFold
from xgboost.sklearn import XGBClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import deepchem as dc
from deepchem.utils import download_url, load_from_disk
from deepchem.utils.vina_utils import prepare_inputs
from deepchem.models import AtomicConvModel
from deepchem.feat import RdkitGridFeaturizer
from joblib import Parallel, delayed
from tqdm import tqdm
import glob
import tempfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, regularizers
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.optimizers import Adadelta, Adam, RMSprop
import hyperopt
from hyperopt import hp, tpe, Trials, fmin, STATUS_OK, space_eval

# Data training and test set #

In [ ]:
#Provide the pathway to the csv training data file:
train_data = pd.read_csv("Provide_the_pathway_to_your_csv_training_data_file")

In [ ]:
#Call the "activity" labels of all training and test molecules:
Train_Class = train_data['activity']

In [ ]:
#Provide the pathway to the csv test data file:
test_data = pd.read_csv("Provide_the_pathway_to_your_csv_test_data_file")

#Call the "activity" labels of all training and test molecules:
Test_Class = test_data['activity']

In [ ]:
#Provide the pathway to the training molecules (mol2 or multi-mol2):
train_mol2 = opd.read_mol2("Provide_the_pathway_to_the_training_molecule")
train_mol2.columns = ['mol', 'mol_name']
train = train_mol2.merge(train_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')
train_mols = train['mol']

In [ ]:
#Provide the pathway to the test molecules (mol2 or multi-mol2):
test_mol2 = opd.read_mol2("Provide_the pathway_to_the_test_molecules")
test_mol2.columns = ['mol', 'mol_name']
test = test_mol2.merge(test_data.drop_duplicates(subset = ['mol_name']), how = 'left', on = 'mol_name')
test_mols = test['mol']

In [ ]:
#Provide the pathway(s) to the training and set target/receptor structure(s):
receptor_train = next(oddt.toolkit.readfile('mol2', 'Provide_the_pathway_to_your_training_receptor_mol2_file'))
receptor_test = next(oddt.toolkit.readfile('mol2', 'Provide_the_pathway_to_your_test_receptor_mol2_file'))

## Function to obtain PLEC fingerprints for training and test set molecules using the same receptor ##

In [ ]:
#Users may keep only one of the following two functions if the training target/receptor structure is the same as the test target/receptor structure
#Users may change the parameters (size, depth_protein, depth_ligand, distance_cutoff) if they wish
def parallel_plec_train(mol):
    feature = PLEC(mol, protein = receptor_train, size = 4092, 
                   depth_protein = 4, depth_ligand = 2,
                   distance_cutoff = 4.5, sparse = False)
    return feature

In [ ]:
def parallel_plec_test(mol):
    feature = PLEC(mol, protein = receptor_test, size = 4092, 
                   depth_protein = 4, depth_ligand = 2,
                   distance_cutoff = 4.5, sparse = False)
    return feature

## Run the function to compute PLEC ##

In [ ]:
#Users may choose another number of cores (num_cores) that corresponds to their computing resources
num_cores = 16
train_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_train)(mol) for mol in tqdm(train_mols))
test_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(parallel_plec_test)(mol) for mol in tqdm(test_mols))

In [ ]:
# Test for identifying [i] if Test_Class has Nan (caution)
nan_indices = [i for i, value in enumerate(Test_Class) if pd.isna(value)]

# Imprimir los valores NaN en Test_Class
print("Índices con NaN en Test_Class:", nan_indices)

# Imprimir los valores correspondientes
print("Valores NaN en Test_Class:")
for i in nan_indices:
    print(f"Índice {i}: {Test_Class[i]}")

## Training and Testing of ML Models Without Optimization but With Cross-Validation ##

## SVM ##

In [ ]:
# Convert classes to numerical format
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = np.array([Activity_Dict[item] for item in Train_Class]).flatten()  # Ensure it's a one-dimensional array
y_test = np.array([Activity_Dict[item] for item in Test_Class]).flatten()    # Ensure it's a one-dimensional array

train_features_array = np.array(train_features)
test_features_array = np.array(test_features)

# Cross-validation configuration (5 folds)
k_folds = 5
skf = StratifiedKFold(n_splits=k_folds, shuffle=True, random_state=42)

# List to store AUC scores per fold
auc_scores = []

for fold, (train_idx, val_idx) in enumerate(skf.split(train_features_array, y_train)):
    print(f"Training fold {fold+1}/{k_folds}...")

    X_train_fold, X_val_fold = train_features_array[train_idx], train_features_array[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Define SVM model
    svm_plec = SVC(degree=3, kernel="rbf", probability=True)

    # Train model on this fold
    svm_plec.fit(X_train_fold, y_train_fold)

    # Evaluate on validation set
    y_val_pred_prob = svm_plec.predict_proba(X_val_fold)[:, 1]  # Probability of the positive class
    auc_val = roc_auc_score(y_val_fold, y_val_pred_prob)
    auc_scores.append(auc_val)

    print(f"Fold {fold+1}: AUC = {auc_val:.4f}")

# Save cross-validation results to CSV
auc_cv_filename = "Provide_the_pathway_to_your_SVM-CrossValidation-AUC.csv"
auc_cv_df = pd.DataFrame({"Fold": range(1, k_folds+1), "AUC": auc_scores})
auc_cv_df.loc["Mean"] = ["Mean", np.mean(auc_scores)]
auc_cv_df.loc["Std"] = ["Std", np.std(auc_scores)]
auc_cv_df.to_csv(auc_cv_filename, index=False)

print(f"Cross-validation results saved to {auc_cv_filename}")

# Train final model on the entire training dataset
svm_plec.fit(train_features_array, y_train)

# Test set evaluation
prediction_test_svm_plec_class = svm_plec.predict(test_features_array)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features_array)

# Compute and save test AUC
auc_test = roc_auc_score(y_test, prediction_test_svm_plec_prob[:, 1])  # Use probability of positive class
auc_test_filename = "Provide_the_pathway_to_your_SVM-Test-AUC.csv"
pd.DataFrame({"AUC": [auc_test]}).to_csv(auc_test_filename, index=False)

print(f"Test set AUC: {auc_test:.4f}")
print(f"Test AUC file saved as {auc_test_filename}")


In [ ]:
# Ensure that Test_Class, prediction_test_svm_plec_class, and prediction_test_svm_plec_prob exist
if 'Test_Class' not in locals() or 'prediction_test_svm_plec_class' not in locals() or 'prediction_test_svm_plec_prob' not in locals():
    raise ValueError("The variables Test_Class, prediction_test_svm_plec_class, or prediction_test_svm_plec_prob are not defined.")

# Define the log file name
log_filename = "Provide_the_pathway_to_your_output_log_SVM.txt"

# Ensure that the OTS directory exists
os.makedirs("Provide_the_pathway_to_your_directory_where_you_save_model", exist_ok=True)

# Open the file to write the output
with open(log_filename, "w") as f:
    try:
        # Verify sizes
        f.write(f"Size of Test_Class: {len(Test_Class)}\n")
        f.write(f"Size of prediction_test_svm_plec_class: {len(prediction_test_svm_plec_class)}\n")

        # Verify contents of Test_Class
        f.write(f"Full content of Test_Class:\n{Test_Class}\n")

        # Create a DataFrame with virtual screening results
        plec_result_svm = pd.DataFrame({
            "Active_Prob": prediction_test_svm_plec_prob[:, 1],
            "Inactive_Prob": prediction_test_svm_plec_prob[:, 0],
            "Predicted_Class": prediction_test_svm_plec_class,
            "Real_Class": Test_Class
        })

        # Verify DataFrame before saving
        f.write(f"First rows of plec_result_svm:\n{plec_result_svm.head()}\n")

        # Save DataFrame to CSV
        csv_filename = "Provide_the_pathway_to_your_SVM_run#.csv"
        plec_result_svm.to_csv(csv_filename, index=False)
        f.write(f"CSV file saved as {csv_filename}\n")


## RF ##

In [ ]:
#Train the RF model on the training molecules:
rf_plec = RandomForestClassifier(n_estimators = 200, max_depth=20, max_features = 'sqrt',  n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

In [ ]:
# Perform cross-validation
scores = cross_val_score(rf_plec, train_features, Train_Class, cv=5, scoring='roc_auc')

# Create a DataFrame with the results
cv_results = pd.DataFrame({
    'Fold': [f'Fold {i+1}' for i in range(len(scores))],
    'AUC': scores
})

# Save the results to a CSV file
cv_results.to_csv("Provide_the_pathway_to_your_cross_validation_RF_results.csv", index=False)

# Print the results
print(f"AUC Cross-Validation: {scores.mean()} ± {scores.std()}")

# Define the log file name
log_filename = "Provide_the_pathway_to_your_output_log.txt"

# Open the file to write the output
with open(log_filename, "w") as f:
    # Verify sizes
    f.write(f"Size of Test_Class: {len(Test_Class)}\n")
    f.write(f"Size of prediction_test_rf_plec_class: {len(prediction_test_rf_plec_class)}\n")
    
    # Verify contents of Test_Class
    f.write(f"Full content of Test_Class:\n{Test_Class}\n")

    # Create DataFrame with virtual screening results
    plec_result_rf = pd.DataFrame({
        "Active_Prob": prediction_test_rf_plec_prob[:, 0],
        "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
        "Predicted_Class": prediction_test_rf_plec_class,
        "Real_Class": Test_Class
    })

    # Verify DataFrame before saving
    f.write(f"First rows of plec_result_rf:\n{plec_result_rf.head()}\n")

    # Save to CSV
    os.makedirs("Provide_the_pathway_to_your_file_where_save_results", exist_ok=True)  # Ensure the OTS directory exists
    csv_filename = "Provide_the_pathway_to_your_file_where_save_results_RF_run#.csv"
    plec_result_rf.to_csv(csv_filename, index=False)
    f.write(f"CSV file saved as {csv_filename}\n")


## XGB ##

In [ ]:
#Train the XGB model on the training molecules:
xgb_plec = XGBClassifier(n_jobs = 40)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

In [ ]:
# Cross-validation evaluation
cv_scores = cross_val_score(xgb_plec, np.array(train_features), Train_Class, cv=5, scoring='roc_auc')

# Print the result
print(f"Cross-validation AUC: {cv_scores.mean()} ± {cv_scores.std()}")

# Save the cross-validation results to a CSV file
cv_results = pd.DataFrame({
    'Fold': range(1, 6),
    'AUC': cv_scores
})

cv_results.to_csv("Provide_the_pathway_to_your_xgb_cross_validation_results.csv", index=False)

print("The cross-validation results have been saved in 'Provide_the_pathway_to_your_xgb_cross_validation_results.csv'.")


In [ ]:
# Verify sizes
f.write(f"Size of Test_Class: {len(Test_Class)}\n")
f.write(f"Size of prediction_test_xgb_plec_class: {len(prediction_test_xgb_plec_class)}\n")

# Verify content of Test_Class
f.write(f"Full content of Test_Class:\n{Test_Class}\n")

# Create DataFrame from virtual screening results of XGB
plec_result_xgb = pd.DataFrame({
    "Active_Prob": prediction_test_xgb_plec_prob[:, 0],
    "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
    "Predicted_Class": prediction_test_xgb_plec_class,
    "Real_Class": Test_Class
})

# Verify DataFrame before saving
f.write(f"First rows of plec_result_xgb:\n{plec_result_xgb.head()}\n")

# Save to CSV
csv_filename = "Provide_the_pathway_to_your_XGB_run#.csv"
plec_result_xgb.to_csv(csv_filename, index=False)
f.write(f"CSV file saved as {csv_filename}\n")


## ANN ##

In [ ]:
# Define the ANN model
ann_plec = MLPClassifier(max_iter=9000)

# Define the cross-validation strategy (5 stratified folds)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Evaluate the model using cross-validation with AUC
cv_scores = cross_val_score(ann_plec, train_features, Train_Class, cv=skf, scoring=make_scorer(roc_auc_score))

# Save the results in a DataFrame and export to CSV
df_results = pd.DataFrame({'Fold': range(1, 6), 'AUC': cv_scores})
df_results.loc[len(df_results)] = ['Mean', np.mean(cv_scores)]
df_results.loc[len(df_results)] = ['Std', np.std(cv_scores)]
df_results.to_csv('Provide_the_pathway_to_your_ANN_cross_val_results.csv', index=False)

# Display the results
print(f'AUC per fold: {cv_scores}')
print(f'Average AUC: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}')

# Train the model on the entire training set
ann_plec.fit(train_features, Train_Class)

# Evaluate the model on the test set
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

# Display test set results
print("Class predictions on the test set:", prediction_test_ann_plec_class)
print("Prediction probabilities on the test set:", prediction_test_ann_plec_prob)


In [ ]:
# Define the name of the output .txt file
log_filename = "Provide_the_pathway_to_your_output_ann_log.txt"

# Ensure the OTS directory exists
os.makedirs("Provide_the_pathway_to_your_file_results", exist_ok=True)

# Open the file to write the output
with open(log_filename, "w") as f:
    # Verify sizes
    f.write(f"Size of Test_Class: {len(Test_Class)}\n")
    f.write(f"Size of prediction_test_ann_plec_class: {len(prediction_test_ann_plec_class)}\n")
    
    # Check content of Test_Class
    f.write(f"Full content of Test_Class:\n{Test_Class}\n")

    # Create DataFrame from virtual screening results 
    plec_result_ann = pd.DataFrame({
        "Active_Prob": prediction_test_ann_plec_prob[:, 0],
        "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
        "Predicted_Class": prediction_test_ann_plec_class,
        "Real_Class": Test_Class
    })

    # Verify DataFrame before saving
    f.write(f"First rows of plec_result_ann:\n{plec_result_ann.head()}\n")

    # Save to CSV
    csv_filename = "Provide_the_pathway_to_your_file_where_you_save_result_ANN-run#.csv"
    plec_result_ann.to_csv(csv_filename, index=False)
    f.write(f"CSV file saved as {csv_filename}\n")


## DNN ##

In [ ]:
#Train the DNN model on the training molecules:
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

dnn_plec = keras.Sequential([
    layers.Dense(8192, kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(0),
    layers.Dense(4069, activation = "relu", kernel_regularizer = regularizers.l2(0.001)),
    layers.BatchNormalization(),
    layers.Dense(2048, activation = "relu"),
    layers.Dropout(0),
    layers.Dense(1, activation = "sigmoid")
])
dnn_plec.compile(optimizer = "rmsprop", loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_plec.fit(np.array(train_features), y_train, epochs = 100, batch_size = 500, verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_plec_prob = dnn_plec.predict(np.array(test_features))
prediction_test_dnn_plec_class = ["Active" if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]


In [ ]:
# Define the name of the output .txt file
log_filename = "Provide_the_pathway_to_your_output_log_DNN_test.txt"

# Open the file to write the output
with open(log_filename, "w") as f:
    # Verify sizes
    f.write(f"Size of Test_Class: {len(Test_Class)}\n")
    f.write(f"Size of prediction_test_dnn_plec_class: {len(prediction_test_dnn_plec_class)}\n")
    
    # Check content of Test_Class
    f.write(f"Full content of Test_Class:\n{Test_Class}\n")

    # Create DataFrame from virtual screening results
    plec_result_dnn_test = pd.DataFrame({
        "Active_Prob": prediction_test_dnn_plec_prob[:, 0],
        "Predicted_Class": prediction_test_dnn_plec_class,
        "Real_Class": Test_Class
    })

    # Verify DataFrame before saving
    f.write(f"First rows of plec_result_dnn_test:\n{plec_result_dnn_test.head()}\n")

    # Ensure the OTS directory exists
    os.makedirs("Provide_the_pathway_to_your_file_where_save_the_results", exist_ok=True)

    # Save to CSV
    csv_filename_test = "Provide_the_pathway_to_your_file_where_save_the_result_DNN-run#.csv"
    plec_result_dnn_test.to_csv(csv_filename_test, index=False)
    f.write(f"CSV file saved as {csv_filename_test}\n")


In [ ]:
# Define the activity dictionary
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = np.array([Activity_Dict[item] for item in Train_Class]).astype("float32")
y_test = np.array([Activity_Dict[item] for item in Test_Class]).astype("float32")

# Set up 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# List to store ROC-AUC values for each fold
roc_auc_scores = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(train_features)):
    print(f"Fold {fold + 1}")
    
    # Split data into training and validation sets for this fold
    X_train, X_val = train_features[train_index], train_features[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Define the DNN model
    tf.random.set_seed(0)
    dnn_plec = keras.Sequential([
        layers.Dense(8192, kernel_regularizer=regularizers.l2(0.001), activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0),
        layers.Dense(4069, activation="relu", kernel_regularizer=regularizers.l2(0.001)),
        layers.BatchNormalization(),
        layers.Dense(2048, activation="relu"),
        layers.Dropout(0),
        layers.Dense(1, activation="sigmoid")
    ])
    
    # Compile the model
    dnn_plec.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])
    
    # Train the model
    dnn_plec.fit(X_train, y_train_fold, epochs=100, batch_size=500, verbose=False)
    
    # Predict probabilities for the positive class (active)
    y_val_pred_prob = dnn_plec.predict(X_val)
    
    # Compute ROC-AUC for this fold
    roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob)
    roc_auc_scores.append(roc_auc)
    print(f"ROC-AUC for Fold {fold + 1}: {roc_auc}")

# Save the ROC-AUC results to a CSV file
roc_auc_results = pd.DataFrame({"Fold": range(1, 6), "ROC_AUC": roc_auc_scores})
roc_auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_ROC_AUC_results", index=False)

# Train the model with all training data to use it on the test set
dnn_plec.fit(train_features, y_train, epochs=100, batch_size=500, verbose=False)

# Test the model on test molecules
prediction_test_dnn_plec_prob = dnn_plec.predict(test_features)
prediction_test_dnn_plec_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

# Save the virtual screening results to a CSV file
plec_result_dnn = pd.DataFrame({
    "Active_Prob": prediction_test_dnn_plec_prob[:, 0],
    "Predicted_Class": prediction_test_dnn_plec_class,
    "Real_Class": Test_Class
})
plec_result_dnn.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run", index=False)


## Training and Testing of ML Models With Optimization for PLEC ##

## SVM ##

In [ ]:
# Define the search space for optimal parameters:
space = {"C": hp.uniform("C", 0, 20),
         "gamma": hp.choice("gamma", ['scale', 'auto']),
         "kernel": hp.choice("kernel", ['rbf', 'poly', 'sigmoid'])}

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_SVM(space):
    with parallel_backend(backend="multiprocessing", n_jobs=40):
        model = SVC(C=space['C'],
                    gamma=space['gamma'],
                    kernel=space['kernel'])
        model.fit(np.array(train_features), Train_Class)
        predicted_train = model.predict(np.array(train_features))
        mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
        
# Search for optimal parameters:
trials = Trials()
best_svm_classification = fmin(fn=hyperparameter_tuning_SVM, space=space, algo=tpe.suggest,
                               max_evals=10, trials=trials)
best_params = space_eval(space, best_svm_classification)

# Optimal parameters:
best_params

# Train the SVM model on the training molecules using optimal parameters:
svm_plec = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'], 
               probability=True, random_state=0)
svm_plec.fit(train_features, Train_Class)

# Test the SVM model on the test molecules:
prediction_test_svm_plec_class = svm_plec.predict(test_features)
prediction_test_svm_plec_prob = svm_plec.predict_proba(test_features)

# Define the name of the .txt file for output logging
log_filename = "output_log_SVM.txt"

# Open the file to write the output
with open(log_filename, "w") as f:
    # Check sizes
    f.write(f"Size of Test_Class: {len(Test_Class)}\n")
    f.write(f"Size of prediction_test_svm_plec_class: {len(prediction_test_svm_plec_class)}\n")
    
    # Check contents of Test_Class
    f.write(f"Full content of Test_Class:\n{Test_Class}\n")

    # Create DataFrame for virtual screening results of the optimized SVM model
    plec_result_svm = pd.DataFrame({
        "Active_Prob": prediction_test_svm_plec_prob[:, 0],
        "Inactive_Prob": prediction_test_svm_plec_prob[:, 1],
        "Predicted_Class": prediction_test_svm_plec_class,
        "Real_Class": Test_Class
    })

    # Verify DataFrame before saving
    f.write(f"First rows of plec_result_svm:\n{plec_result_svm.head()}\n")

    # Save to CSV
    csv_filename = "Provide_the_pathway_to_your_file_where_save_the_result_run.csv"
    plec_result_svm.to_csv(csv_filename, index=False)
    f.write(f"CSV file saved as {csv_filename}\n")


## RF ##

In [ ]:
#Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 100, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy'])}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(n_estimators = int(space['n_estimators']),
                                   max_depth = space['max_depth'],
                                   criterion = space['criterion'], n_jobs = 40)
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn = hyperparameter_tuning_randomforest, space = space, algo = tpe.suggest,
                              max_evals = 10, trials = trials)
best_params = space_eval(space, best_rf_classification)

#Optimal parameters:
best_params
#Train the RF model on the training molecules, using optimal parameters:
rf_plec = RandomForestClassifier(n_estimators = int(best_params['n_estimators']), 
                                 max_depth = best_params['max_depth'], 
                                 criterion = best_params['criterion'],
                                 max_features = 'sqrt', n_jobs = 30)
rf_plec.fit(train_features, Train_Class)

#Test the RF model on the test molecules:
prediction_test_rf_plec_class = rf_plec.predict(test_features)
prediction_test_rf_plec_prob = rf_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_plec_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_plec_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_plec_class,
                               "Real_Class": Test_Class})
plec_result_rf.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")

## XGB ##

In [ ]:
#Define the search space for optimal parameters:
space = {"max_depth": hp.uniform("max_depth", 3, 18),
         "gamma": hp.uniform("gamma", 1, 9),
         "reg_alpha": hp.uniform("reg_alpha", 40, 180),
         "reg_lambda": hp.uniform("reg_lambda", 0, 1), 
         "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
         "min_child_weight": hp.uniform("min_child_weight", 0, 10),
         "n_estimators": hp.uniform("n_estimators", 1000, 5000)} 

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_XGB(space):
    model = XGBClassifier(objective = "binary:logistic", 
                          max_depth = int(space['max_depth']),
                          gamma = int(space["gamma"]),
                          reg_alpha = int(space["reg_alpha"]),
                          reg_lambda = space['reg_lambda'],
                          colsample_bytree = space["colsample_bytree"],
                          min_child_weight = int(space["min_child_weight"]),
                          n_estimators = int(space['n_estimators']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_xgb_classification = fmin(fn = hyperparameter_tuning_XGB, space = space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_xgb_classification)

#Optimal parameters:
best_params
#Train the XGB model on the training molecules, using optimal parameters:
xgb_plec = XGBClassifier(objective = "binary:logistic",
                         max_depth = int(best_params['max_depth']),
                         gamma = int(best_params['gamma']),
                         reg_alpha = int(best_params['reg_alpha']),
                         reg_lambda = best_params['reg_lambda'],
                         colsample_bytree = best_params['colsample_bytree'],
                         min_child_weight = int(best_params['min_child_weight']),
                         n_estimators = int(best_params['n_estimators']),
                         n_jobs = 40, random_state = 0)
xgb_plec.fit(np.array(train_features), Train_Class)

#Test the XGB model on the test molecules:
prediction_test_xgb_plec_class = xgb_plec.predict(np.array(test_features))
prediction_test_xgb_plec_prob = xgb_plec.predict_proba(np.array(test_features))

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_xgb = pd.DataFrame({"Active_Prob": prediction_test_xgb_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_xgb_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_xgb_plec_class,
                                "Real_Class": Test_Class})
plec_result_xgb.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")

## ANN ##

In [ ]:
#Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_features), Train_Class)
    predicted_train = model.predict(np.array(train_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
#Search for optimal parameters:
trials = Trials()
best_ann_classification = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classification)

#Optimal parameters:
best_params
#Train the ANN model on the training molecules, using optimal parameters:
ann_plec = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_plec.fit(train_features, Train_Class)

#Test the ANN model on the test molecules:
prediction_test_ann_plec_class = ann_plec.predict(test_features)
prediction_test_ann_plec_prob = ann_plec.predict_proba(test_features)

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_plec_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_plec_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_plec_class,
                                "Real_Class": Test_Class})
plec_result_ann.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")

## DNN ##

In [ ]:
#Define the search space for optimal parameters:
space = {'choice': hp.choice('num_layers',
                            [ {'layers': 'two'},
                              {'layers': 'three',
                               'units4': hp.uniform('units4', 64, 8192), 
                               'dropout3': hp.uniform('dropout3', 0, 1)}
                            ]),
        'units1': hp.uniform('units1', 64, 8192),
        'units2': hp.uniform('units2', 64, 8192),
        'units3': hp.uniform('units3', 64, 8192),
        'dropout1': hp.uniform('dropout1', 0, 1),
        'dropout2': hp.uniform('dropout2', 0, 1),
        'batch_size': hp.uniform('batch_size', 128, 500),
        'nb_epochs': 100,
        'optimizer': hp.choice('optimizer', ['Adadelta','Adam','rmsprop']),
        'activation': 'relu'
        }
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

#Define the function for hyperparameter tuning:
def hyperparameter_tuning_DNN(space):
    model = keras.Sequential([
        layers.Dense(abs(int(space['units1'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dropout(space['dropout1']),
        layers.Dense(abs(int(space['units2'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dense(abs(int(space['units3'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.Dropout(space['dropout2']),
        layers.Dense(1, activation = "sigmoid")
    ])
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(np.array(train_features), y_train, epochs = 100, batch_size = int(space['batch_size']), verbose = False)
    predict_proba = model.predict(np.array(train_features))[:, 0]
    predicted_train = ['Active' if num >= 0.5 else "Inactive" for num in predict_proba]
    #Calculate Mattehews Correlation Coefficient
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model, 'MCC': mcc}

#Search for optimal parameters:
trials = Trials()
best_dnn_classification = fmin(hyperparameter_tuning_DNN, space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_dnn_classification)

#Optimal parameters:
best_params

#Train the DNN model on the training molecules, using optimal parameters:
tf.random.set_seed(0)
dnn_plec = keras.Sequential([
    layers.Dense(units = int(best_params['units1']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(best_params['dropout1']),
    layers.Dense(units = int(best_params['units2']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dense(units = int(best_params['units3']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.Dropout(best_params['dropout2']),
    layers.Dense(1, activation = "sigmoid")    
])
dnn_plec.compile(optimizer = best_params['optimizer'], loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_plec.fit(np.array(train_features), y_train, epochs = 100, 
             batch_size = int(best_params['batch_size']), verbose = False)

#Test the DNN model on the test molecules:
prediction_test_dnn_plec_prob = dnn_plec.predict(np.array(test_features))
prediction_test_dnn_plec_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_plec_prob]

#Get virtual screening results on the test molecules and export results to a csv file:
plec_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_plec_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_plec_class,
                                "Real_Class": Test_Class})
plec_result_dnn.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")


## Run the function to compute Grid ##

In [ ]:
# Provide the pathway to the training molecules (SDF or multi-SDF)
train_sdf = opd.read_sdf("Provide_the_pathway_to_your_training_molecules.sdf")
train = train_sdf.merge(train_data.drop_duplicates(subset=['mol_name']), how='left', on='mol_name')

# Split the multi-SDF training data into individual SDF structures
for i in range(train_sdf.shape[0]):
    each_sdf = train_sdf.iloc[[i]]
    each_name = list(each_sdf['mol_name'])[0]
    out_path = "Provide_the_directory_where_you_want_to_save_individual_training_SDFs/" + str(each_name) + ".sdf"
    ChemDataFrame.to_sdf(each_sdf, out_path)
training_set = glob.glob("Provide_the_directory_where_you_saved_individual_training_SDFs/*")

# Provide the pathway to the test molecules (SDF or multi-SDF)
test_sdf = opd.read_sdf("Provide_the_pathway_to_your_test_molecules.sdf")
test = test_sdf.merge(test_data.drop_duplicates(subset=['mol_name']), how='left', on='mol_name')

# Split the multi-SDF test data into individual SDF structures
for i in range(test_sdf.shape[0]):
    each_sdf = test_sdf.iloc[[i]]
    each_name = list(each_sdf['mol_name'])[0]
    out_path = "Provide_the_directory_where_you_want_to_save_individual_test_SDFs/" + str(each_name) + ".sdf"
    ChemDataFrame.to_sdf(each_sdf, out_path)
test_set = glob.glob("Provide_the_directory_where_you_saved_individual_test_SDFs/*")

# Extract the structures of all training and test molecules
train_mols = train['mol']
test_mols = test['mol']

# Provide the pathway(s) to the training and test target/receptor structure(s)
protein_train = "Provide_the_pathway_to_your_training_protein_structure.pdb"
protein_test = "Provide_the_pathway_to_your_test_protein_structure.pdb"


In [ ]:
#Users may keep only one of the following two functions if the training target/receptor structure is the same as the test target/receptor structure
#Users may change the parameters (voxel_width, feature_types, ecfp_power, splif_power) if they wish
featurizer = RdkitGridFeaturizer(voxel_width = 16.0, feature_types = ["ecfp", "splif", "hbond", "salt_bridge"], ecfp_power = 9, splif_power = 9, flatten = True, verbose = False)
def extract_grid_feature_train(ligand_file):
    try:
        feature = featurizer._featurize((ligand_file, protein_train))
        return feature
    except:
        pass
def extract_grid_feature_test(ligand_file):
    try:
        feature = featurizer._featurize((ligand_file, protein_test))
        return feature
    except:
        pass

#Users may choose another number of cores (num_cores) that corresponds to their computing resources
num_cores = 20
train_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(extract_grid_feature_train)(ligand_file) for ligand_file in tqdm(training_set))
test_features = Parallel(n_jobs = num_cores, backend = "multiprocessing")(delayed(extract_grid_feature_test)(ligand_file) for ligand_file in tqdm(test_set))

#Export grid features as npy files:
np.save('features_grid_train.npy', train_features)

test_features = np.array(test_features, dtype=np.float32)  # Asegurar una estructura homogénea
np.save('features_grid_test.npy', test_features)


#Load grid features for the next stages:
train_grid_features = np.load("features_grid_train.npy", allow_pickle=True)
#test_grid_features = np.load("features_grid_test.npy", allow_pickle=True)
test_grid_features = np.load("features_grid_test.npy", allow_pickle=False)  # Evita dtype=object
test_grid_features = np.vstack(test_grid_features)


In [ ]:
# Verify the number of molecules at each step
training_count = len(training_set)
grid_features_count = len(train_grid_features)
train_class_count = len(Train_Class)

# Save sizes in CSV
debug_sizes = pd.DataFrame({
    "Variable": ["training_set", "train_grid_features", "Train_Class"],
    "Length": [training_count, grid_features_count, train_class_count]
})
debug_sizes.to_csv("debug_sizes.csv", index=False)

# Check for duplicate files in training_set
file_counts = pd.Series(training_set).value_counts()
duplicates = file_counts[file_counts > 1]

# Create dataframe with duplicate information
duplicates_df = pd.DataFrame({
    "Filename": duplicates.index,
    "Occurrences": duplicates.values
})
duplicates_df.to_csv("duplicates_check.csv", index=False)

# Check for extra files in training_set compared to Train_Class
training_filenames = [f.split("/")[-1].replace(".sdf", "") for f in training_set]
train_class_names = list(train['mol_name'])  # Ensure the names match

extra_files = set(training_filenames) - set(train_class_names)

# Save extra files in CSV
extra_files_df = pd.DataFrame({"Extra_Files": list(extra_files)})
extra_files_df.to_csv("extra_files.csv", index=False)

# Trim train_grid_features if it has more elements
if grid_features_count > train_class_count:
    train_grid_features = train_grid_features[:train_class_count]
    print("✅ Adjusted: train_grid_features now matches Train_Class")

# Ensure train_grid_features is a properly formatted matrix for sklearn
train_grid_features_fixed = np.array([np.array(f).flatten() if isinstance(f, (list, np.ndarray)) else np.full((1,), f) for f in train_grid_features])

# Save the corrected array to files
np.save("train_grid_features_fixed.npy", train_grid_features_fixed)
train_grid_features_df = pd.DataFrame(train_grid_features_fixed)
train_grid_features_df.to_csv("train_grid_features_fixed.csv", index=False)

# Load grid features for the next stages
train_grid_features = np.load("train_grid_features_fixed.npy", allow_pickle=True)
test_grid_features = np.load("features_grid_test.npy", allow_pickle=True)

print("✅ The following files have been generated:")
print("- debug_sizes.csv → Size information")
print("- duplicates_check.csv → Duplicate file information")
print("- extra_files.csv → List of extra files in training_set")
print("- train_grid_features_fixed.npy → Corrected train_grid_features in numpy format")
print("- train_grid_features_fixed.csv → Corrected train_grid_features in CSV format")


In [ ]:
import numpy as np
import pandas as pd

# Verify the number of molecules at each step
training_count = len(training_set)
grid_features_count = len(train_grid_features)
train_class_count = len(Train_Class)

# Save sizes in CSV
debug_sizes = pd.DataFrame({
    "Variable": ["training_set", "train_grid_features", "Train_Class"],
    "Length": [training_count, grid_features_count, train_class_count]
})
debug_sizes.to_csv("debug_sizes.csv", index=False)

# Check for duplicate files in training_set
file_counts = pd.Series(training_set).value_counts()
duplicates = file_counts[file_counts > 1]

# Create dataframe with duplicate information
duplicates_df = pd.DataFrame({
    "Filename": duplicates.index,
    "Occurrences": duplicates.values
})
duplicates_df.to_csv("duplicates_check.csv", index=False)

# Check for extra files in training_set compared to Train_Class
training_filenames = [f.split("/")[-1].replace(".sdf", "") for f in training_set]
train_class_names = list(train['mol_name'])  # Ensure the names match

extra_files = set(training_filenames) - set(train_class_names)

# Save extra files in CSV
extra_files_df = pd.DataFrame({"Extra_Files": list(extra_files)})
extra_files_df.to_csv("extra_files.csv", index=False)

# Trim train_grid_features if it has more elements
if grid_features_count > train_class_count:
    train_grid_features = train_grid_features[:train_class_count]
    print("✅ Adjusted: train_grid_features now matches Train_Class")

# Ensure train_grid_features is a properly formatted matrix for sklearn
train_grid_features_fixed = np.array([np.array(f).flatten() if isinstance(f, (list, np.ndarray)) else np.full((1,), f) for f in train_grid_features])

# Save the corrected array to files
np.save("train_grid_features_fixed.npy", train_grid_features_fixed)
train_grid_features_df = pd.DataFrame(train_grid_features_fixed)
train_grid_features_df.to_csv("train_grid_features_fixed.csv", index=False)

# Load grid features for the next stages
train_grid_features = np.load("train_grid_features_fixed.npy", allow_pickle=True)
test_grid_features = np.load("features_grid_test.npy", allow_pickle=True)

# Debugging test_grid_features
debug_info = []

debug_info.append(["Train features shape", train_grid_features.shape])
debug_info.append(["Test features shape", test_grid_features.shape])

if isinstance(test_grid_features, pd.DataFrame):
    dtypes_info = test_grid_features.dtypes.astype(str).to_dict()
    debug_info.append(["Test feature types", dtypes_info])
    if any(test_grid_features.dtypes == 'object'):
        test_grid_features = test_grid_features.astype(float)
else:
    debug_info.append(["Test feature dtype", str(test_grid_features.dtype)])

nested_list_issues = []
for i, row in enumerate(test_grid_features):
    if isinstance(row, list) or isinstance(row, np.ndarray):
        nested_list_issues.append(f"Row {i} contains a nested list instead of individual values.")

if nested_list_issues:
    debug_info.append(["Nested list issues", nested_list_issues])

row_lengths = [len(row) if isinstance(row, (list, np.ndarray)) else 1 for row in test_grid_features]
max_row_length = max(row_lengths)

if any(length != max_row_length for length in row_lengths):
    debug_info.append(["Inconsistent row lengths", row_lengths])

test_grid_features = np.array([np.ravel(row) if isinstance(row, (list, np.ndarray)) else row for row in test_grid_features])

df_debug = pd.DataFrame(debug_info, columns=["Description", "Value"])
df_debug.to_csv("debugging_output-test.csv", index=False)

print("✅ The following files have been generated:")
print("- debug_sizes.csv → Size information")
print("- duplicates_check.csv → Duplicate file information")
print("- extra_files.csv → List of extra files in training_set")
print("- train_grid_features_fixed.npy → Corrected train_grid_features in numpy format")
print("- train_grid_features_fixed.csv → Corrected train_grid_features in CSV format")
print("- debugging_output-test.csv → Debugging information for test_grid_features")


## Training and Testing of ML Models Without Optimization but With Cross-Validation ##

## SVM ##

In [ ]:
# Define the SVM model
svm_grid = SVC(degree=3, kernel="rbf", probability=True, random_state=0)

# Configure 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# List to store the AUC-ROC results for each fold
auc_scores = []

# Perform cross-validation
for fold, (train_idx, val_idx) in enumerate(cv.split(train_grid_features, Train_Class)):
    X_train, X_val = train_grid_features[train_idx], train_grid_features[val_idx]
    y_train, y_val = Train_Class[train_idx], Train_Class[val_idx]
    
    # Train the model on the training fold
    svm_grid.fit(X_train, y_train)
    
    # Predict probabilities on the validation fold
    y_val_prob = svm_grid.predict_proba(X_val)[:, 1]
    
    # Calculate AUC-ROC for this fold
    auc = roc_auc_score(y_val, y_val_prob)
    auc_scores.append(auc)
    
    print(f"Fold {fold + 1}: AUC-ROC = {auc:.4f}")

# Save AUC-ROC results to a CSV file
auc_results = pd.DataFrame({"Fold": range(1, 6), "AUC-ROC": auc_scores})
auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)

# Train the final model on the entire training set
svm_grid.fit(train_grid_features, Train_Class)

# Predict on the test set
prediction_test_svm_grid_class = svm_grid.predict(test_grid_features)
prediction_test_svm_grid_prob = svm_grid.predict_proba(test_grid_features)

# Save prediction results to a CSV file
grid_result_svm = pd.DataFrame({
    "Active_Prob": prediction_test_svm_grid_prob[:, 0],
    "Inactive_Prob": prediction_test_svm_grid_prob[:, 1],
    "Predicted_Class": prediction_test_svm_grid_class,
    "Real_Class": Test_Class
})
grid_result_svm.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## RF ##

In [ ]:
# Define the Random Forest model with specified parameters
rf_grid = RandomForestClassifier(n_estimators=500, n_jobs=30, random_state=0)

# Configure 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# List to store ROC-AUC values for each fold
roc_auc_scores = []

# Perform cross-validation
for train_index, val_index in kf.split(train_grid_features):
    # Split data into training and validation for this fold
    X_train, X_val = train_grid_features[train_index], train_grid_features[val_index]
    y_train, y_val = Train_Class[train_index], Train_Class[val_index]
    
    # Train the model
    rf_grid.fit(X_train, y_train)
    
    # Predict probabilities for the positive class (active)
    y_val_pred_prob = rf_grid.predict_proba(X_val)[:, 1]
    
    # Calculate ROC-AUC for this fold
    roc_auc = roc_auc_score(y_val, y_val_pred_prob)
    roc_auc_scores.append(roc_auc)

# Save ROC-AUC results to a CSV file
roc_auc_results = pd.DataFrame({"Fold": range(1, 6), "ROC_AUC": roc_auc_scores})
roc_auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)

# Train the model on the entire training set for testing
rf_grid.fit(train_grid_features, Train_Class)

# Test the model on the test molecules
prediction_test_rf_grid_class = rf_grid.predict(test_grid_features)
prediction_test_rf_grid_prob = rf_grid.predict_proba(test_grid_features)

# Save virtual screening results to a CSV file
grid_result_rf = pd.DataFrame({
    "Active_Prob": prediction_test_rf_grid_prob[:, 0],
    "Inactive_Prob": prediction_test_rf_grid_prob[:, 1],
    "Predicted_Class": prediction_test_rf_grid_class,
    "Real_Class": Test_Class
})
grid_result_rf.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## XGB ##

In [ ]:
# Define the XGBoost model with optimized parameters
xgb_grid = XGBClassifier(n_estimators=745, max_depth=7, learning_rate=0.05, n_jobs=40, random_state=0)

# Configure 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# List to store ROC-AUC values for each fold
roc_auc_scores = []

# Perform cross-validation
for train_index, val_index in kf.split(train_grid_features):
    # Split data into training and validation for this fold
    X_train, X_val = train_grid_features[train_index], train_grid_features[val_index]
    y_train, y_val = Train_Class[train_index], Train_Class[val_index]
    
    # Train the model
    xgb_grid.fit(X_train, y_train)
    
    # Predict probabilities for the positive class (active)
    y_val_pred_prob = xgb_grid.predict_proba(X_val)[:, 1]
    
    # Calculate ROC-AUC for this fold
    roc_auc = roc_auc_score(y_val, y_val_pred_prob)
    roc_auc_scores.append(roc_auc)

# Save ROC-AUC results to a CSV file
roc_auc_results = pd.DataFrame({"Fold": range(1, 6), "ROC_AUC": roc_auc_scores})
roc_auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)

# Train the model on the entire training set for testing
xgb_grid.fit(np.array(train_grid_features), Train_Class)

# Test the model on the test molecules
prediction_test_xgb_grid_class = xgb_grid.predict(np.array(test_grid_features))
prediction_test_xgb_grid_prob = xgb_grid.predict_proba(np.array(test_grid_features))

# Save virtual screening results to a CSV file
grid_result_xgb = pd.DataFrame({
    "Active_Prob": prediction_test_xgb_grid_prob[:, 0],
    "Inactive_Prob": prediction_test_xgb_grid_prob[:, 1],
    "Predicted_Class": prediction_test_xgb_grid_class,
    "Real_Class": Test_Class
})
grid_result_xgb.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## ANN ##

In [ ]:
# Define the ANN model
ann_grid = MLPClassifier(max_iter=500, random_state=0)

# Configure 5-fold stratified cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# List to store AUC-ROC results for each fold
auc_scores = []

# Perform cross-validation
for fold, (train_idx, val_idx) in enumerate(cv.split(train_grid_features, Train_Class)):
    X_train, X_val = train_grid_features[train_idx], train_grid_features[val_idx]
    y_train, y_val = Train_Class[train_idx], Train_Class[val_idx]
    
    # Train the model on the training fold
    ann_grid.fit(X_train, y_train)
    
    # Predict probabilities on the validation fold
    y_val_prob = ann_grid.predict_proba(X_val)[:, 1]
    
    # Compute AUC-ROC for this fold
    auc = roc_auc_score(y_val, y_val_prob)
    auc_scores.append(auc)
    
    print(f"Fold {fold + 1}: AUC-ROC = {auc:.4f}")

# Save AUC-ROC results to a CSV file
auc_results = pd.DataFrame({"Fold": range(1, 6), "AUC-ROC": auc_scores})
auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run_CV.csv", index=False)

# Train the final model on the entire training set
ann_grid.fit(train_grid_features, Train_Class)

# Predict on the test set
prediction_test_ann_grid_class = ann_grid.predict(test_grid_features)
prediction_test_ann_grid_prob = ann_grid.predict_proba(test_grid_features)

# Save prediction results to a CSV file
grid_result_ann = pd.DataFrame({
    "Active_Prob": prediction_test_ann_grid_prob[:, 0],
    "Inactive_Prob": prediction_test_ann_grid_prob[:, 1],
    "Predicted_Class": prediction_test_ann_grid_class,
    "Real_Class": Test_Class
})
grid_result_ann.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## DNN ##

In [ ]:
# Define the activity dictionary
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = np.array([Activity_Dict[item] for item in Train_Class]).astype("float32")
y_test = np.array([Activity_Dict[item] for item in Test_Class]).astype("float32")

# Configure 5-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=0)

# List to store ROC-AUC values for each fold
roc_auc_scores = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(train_grid_features)):
    print(f"Fold {fold + 1}")
    
    # Split data into training and validation for this fold
    X_train, X_val = train_grid_features[train_index], train_grid_features[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Define the DNN model
    tf.random.set_seed(0)
    dnn_grid = keras.Sequential([
        layers.Dense(8192, kernel_regularizer=regularizers.l2(0), activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(0),
        layers.Dense(4069, activation="relu", kernel_regularizer=regularizers.l2(0)),
        layers.BatchNormalization(),
        layers.Dense(2048, activation="relu"),
        layers.Dropout(0),
        layers.Dense(1, activation="sigmoid")
    ])
    
    # Compile the model
    dnn_grid.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=['accuracy'])
    
    # Train the model
    dnn_grid.fit(X_train, y_train_fold, epochs=100, batch_size=500, verbose=False)
    
    # Predict probabilities for the positive (active) class
    y_val_pred_prob = dnn_grid.predict(X_val)
    
    # Compute ROC-AUC for this fold
    roc_auc = roc_auc_score(y_val_fold, y_val_pred_prob)
    roc_auc_scores.append(roc_auc)
    print(f"ROC-AUC for Fold {fold + 1}: {roc_auc}")

# Save ROC-AUC results to a CSV file
roc_auc_results = pd.DataFrame({"Fold": range(1, 6), "ROC_AUC": roc_auc_scores})
roc_auc_results.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run_CV.csv", index=False)

# Train the model with all training data for testing
dnn_grid.fit(train_grid_features, y_train, epochs=100, batch_size=500, verbose=False)

# Test the model on test molecules
prediction_test_dnn_grid_prob = dnn_grid.predict(test_grid_features)
prediction_test_dnn_grid_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_grid_prob]

# Save virtual screening results to a CSV file
grid_result_dnn = pd.DataFrame({
    "Active_Prob": prediction_test_dnn_grid_prob[:, 0],
    "Predicted_Class": prediction_test_dnn_grid_class,
    "Real_Class": Test_Class
})
grid_result_dnn.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## Training and Testing of ML Models With Optimization for Grid ##

## SVM ##

In [ ]:
# Define the search space for optimal parameters:
space = {"C": hp.uniform("C", 0, 20),
         "gamma": hp.choice("gamma", ['scale', 'auto']),
         "kernel": hp.choice("kernel", ['rbf', 'poly', 'sigmoid'])}

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_SVM(space):
    with parallel_backend(backend="multiprocessing", n_jobs=40):
        model = SVC(C=space['C'],
                    gamma=space['gamma'],
                    kernel=space['kernel'])
        model.fit(np.array(train_grid_features), Train_Class)
        predicted_train = model.predict(np.array(train_grid_features))
        mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
        
# Search for optimal parameters:
trials = Trials()
best_svm_classification = fmin(fn=hyperparameter_tuning_SVM, space=space, algo=tpe.suggest,
                               max_evals=10, trials=trials)
best_params = space_eval(space, best_svm_classification)

# Optimal parameters:
best_params

# Train the SVM model on the training molecules using optimal parameters:
svm_grid = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'], 
               probability=True, random_state=0)
svm_grid.fit(train_grid_features, Train_Class)

# Test the SVM model on the test molecules:
prediction_test_svm_grid_class = svm_grid.predict(test_grid_features)
prediction_test_svm_grid_prob = svm_grid.predict_proba(test_grid_features)

# Define the name of the .txt file for output logging
log_filename = "output_log_SVM.txt"

# Open the file to write the output
with open(log_filename, "w") as f:
    # Check sizes
    f.write(f"Size of Test_Class: {len(Test_Class)}\n")
    f.write(f"Size of prediction_test_svm_grid_class: {len(prediction_test_svm_grid_class)}\n")
    
    # Check contents of Test_Class
    f.write(f"Full content of Test_Class:\n{Test_Class}\n")

    # Create DataFrame for virtual screening results of the optimized SVM model
    grid_result_svm = pd.DataFrame({
        "Active_Prob": prediction_test_svm_grid_prob[:, 0],
        "Inactive_Prob": prediction_test_svm_grid_prob[:, 1],
        "Predicted_Class": prediction_test_svm_grid_class,
        "Real_Class": Test_Class
    })

    # Verify DataFrame before saving
    f.write(f"First rows of grid_result_svm:\n{grid_result_svm.head()}\n")

    # Save to CSV
    csv_filename = "Provide_the_pathway_to_your_file_where_save_the_result_run.csv"
    grid_result_svm.to_csv(csv_filename, index=False)
    f.write(f"CSV file saved as {csv_filename}\n")


## RF ##

In [ ]:
# Define the search space for optimal parameters:
space = {"n_estimators": hp.uniform("n_estimators", 100, 10000),
         "max_depth": hp.choice("max_depth", [1, 2, 3, 4, 5, None]),
         "criterion": hp.choice("criterion", ['gini', 'entropy'])}

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_randomforest(space):
    model = RandomForestClassifier(n_estimators=int(space['n_estimators']),
                                   max_depth=space['max_depth'],
                                   criterion=space['criterion'], n_jobs=40)
    model.fit(np.array(train_grid_features), Train_Class)
    predicted_train = model.predict(np.array(train_grid_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
# Search for optimal parameters:
trials = Trials()
best_rf_classification = fmin(fn=hyperparameter_tuning_randomforest, space=space, algo=tpe.suggest,
                              max_evals=10, trials=trials)
best_params = space_eval(space, best_rf_classification)

# Optimal parameters:
best_params

# Train the RF model on the training molecules, using optimal parameters:
rf_grid = RandomForestClassifier(n_estimators=int(best_params['n_estimators']), 
                                 max_depth=best_params['max_depth'], 
                                 criterion=best_params['criterion'],
                                 max_features='sqrt', n_jobs=30)
rf_grid.fit(train_grid_features, Train_Class)

# Test the RF model on the test molecules:
prediction_test_rf_grid_class = rf_grid.predict(test_grid_features)
prediction_test_rf_grid_prob = rf_grid.predict_proba(test_grid_features)

# Get virtual screening results on the test molecules and export results to a csv file:
grid_result_rf = pd.DataFrame({"Active_Prob": prediction_test_rf_grid_prob[:, 0],
                               "Inactive_Prob": prediction_test_rf_grid_prob[:, 1],
                               "Predicted_Class": prediction_test_rf_grid_class,
                               "Real_Class": Test_Class})
grid_result_rf.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")


## XGB ##

In [ ]:
# Define the search space for optimal parameters:
space = {
    "max_depth": hp.uniform("max_depth", 3, 18),
    "gamma": hp.uniform("gamma", 1, 9),
    "reg_alpha": hp.uniform("reg_alpha", 40, 180),
    "reg_lambda": hp.uniform("reg_lambda", 0, 1), 
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "min_child_weight": hp.uniform("min_child_weight", 0, 10),
    "n_estimators": hp.uniform("n_estimators", 1000, 5000)
} 

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_XGB(space):
    model = XGBClassifier(
        objective="binary:logistic", 
        max_depth=int(space['max_depth']),
        gamma=int(space["gamma"]),
        reg_alpha=int(space["reg_alpha"]),
        reg_lambda=space['reg_lambda'],
        colsample_bytree=space["colsample_bytree"],
        min_child_weight=int(space["min_child_weight"]),
        n_estimators=int(space['n_estimators']),
        n_jobs=40,
        random_state=0
    )
    model.fit(np.array(train_grid_features), Train_Class)
    predicted_train = model.predict(np.array(train_grid_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1 - mcc, 'status': STATUS_OK, 'model': model}
    
# Search for optimal parameters:
trials = Trials()
best_xgb_classification = fmin(
    fn=hyperparameter_tuning_XGB, 
    space=space, 
    algo=tpe.suggest, 
    max_evals=10, 
    trials=trials
)
best_params = space_eval(space, best_xgb_classification)

# Train the XGB model on the training molecules, using optimal parameters:
xgb_grid = XGBClassifier(
    objective="binary:logistic",
    max_depth=int(best_params['max_depth']),
    gamma=int(best_params['gamma']),
    reg_alpha=int(best_params['reg_alpha']),
    reg_lambda=best_params['reg_lambda'],
    colsample_bytree=best_params['colsample_bytree'],
    min_child_weight=int(best_params['min_child_weight']),
    n_estimators=int(best_params['n_estimators']),
    n_jobs=40, 
    random_state=0
)
xgb_grid.fit(np.array(train_grid_features), Train_Class)

# Test the XGB model on the test molecules:
prediction_test_xgb_grid_class = xgb_grid.predict(np.array(test_grid_features))
prediction_test_xgb_grid_prob = xgb_grid.predict_proba(np.array(test_grid_features))

# Get virtual screening results on the test molecules and export results to a CSV file:
grid_result_xgb = pd.DataFrame({
    "Active_Prob": prediction_test_xgb_grid_prob[:, 0],
    "Inactive_Prob": prediction_test_xgb_grid_prob[:, 1],
    "Predicted_Class": prediction_test_xgb_grid_class,
    "Real_Class": Test_Class
})
grid_result_xgb.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv", index=False)


## ANN ##

In [ ]:
# Define the search space for optimal parameters:
space = {"hidden_layer_sizes": hp.uniform("hidden_layer_sizes", 8, 140),
         "activation": hp.choice("activation", ['relu', 'tanh']),
         "max_iter": hp.uniform("max_iter", 1000, 10000)}

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_ANN(space):
    model = MLPClassifier(hidden_layer_sizes = int(space['hidden_layer_sizes']),
                          activation = space['activation'],
                          max_iter = int(space['max_iter']))
    model.fit(np.array(train_grid_features), Train_Class)
    predicted_train = model.predict(np.array(train_grid_features))
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model}
    
# Search for optimal parameters:
trials = Trials()
best_ann_classification = fmin(fn = hyperparameter_tuning_ANN, space = space, algo = tpe.suggest,
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_ann_classification)

# Optimal parameters:
best_params
# Train the ANN model on the training molecules, using optimal parameters:
ann_grid = MLPClassifier(hidden_layer_sizes = int(best_params['hidden_layer_sizes']), 
                         activation = best_params['activation'], 
                         max_iter = int(best_params['max_iter']), 
                         random_state = 0)
ann_grid.fit(train_grid_features, Train_Class)

# Test the ANN model on the test molecules:
prediction_test_ann_grid_class = ann_grid.predict(test_grid_features)
prediction_test_ann_grid_prob = ann_grid.predict_proba(test_grid_features)

# Get virtual screening results on the test molecules and export results to a CSV file:
grid_result_ann = pd.DataFrame({"Active_Prob": prediction_test_ann_grid_prob[:, 0],
                                "Inactive_Prob": prediction_test_ann_grid_prob[:, 1],
                                "Predicted_Class": prediction_test_ann_grid_class,
                                "Real_Class": Test_Class})
grid_result_ann.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")


## DNN ##

In [ ]:
# Define the search space for optimal parameters:
space = {'choice': hp.choice('num_layers',
                            [ {'layers': 'two'},
                              {'layers': 'three',
                               'units4': hp.uniform('units4', 64, 8192), 
                               'dropout3': hp.uniform('dropout3', 0, 1)}
                            ]),
        'units1': hp.uniform('units1', 64, 8192),
        'units2': hp.uniform('units2', 64, 8192),
        'units3': hp.uniform('units3', 64, 8192),
        'dropout1': hp.uniform('dropout1', 0, 1),
        'dropout2': hp.uniform('dropout2', 0, 1),
        'batch_size': hp.uniform('batch_size', 128, 500),
        'nb_epochs': 100,
        'optimizer': hp.choice('optimizer', ['Adadelta','Adam','rmsprop']),
        'activation': 'relu'
        }
Activity_Dict = {"Active": 1, "Inactive": 0}
y_train = [Activity_Dict[item] for item in Train_Class]
y_test = [Activity_Dict[item] for item in Test_Class]
y_train = np.asarray(y_train).astype("float32")
y_test = np.asarray(y_test).astype("float32")

# Define the function for hyperparameter tuning:
def hyperparameter_tuning_DNN(space):
    model = keras.Sequential([
        layers.Dense(abs(int(space['units1'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dropout(space['dropout1']),
        layers.Dense(abs(int(space['units2'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.BatchNormalization(),
        layers.Dense(abs(int(space['units3'])), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
        layers.Dropout(space['dropout2']),
        layers.Dense(1, activation = "sigmoid")
    ])
    model.compile(optimizer = 'adam', loss = "binary_crossentropy", metrics = ['accuracy'])
    model.fit(np.array(train_grid_features), y_train, epochs = 100, batch_size = int(space['batch_size']), verbose = False)
    predict_proba = model.predict(np.array(train_grid_features))[:, 0]
    predicted_train = ['Active' if num >= 0.5 else "Inactive" for num in predict_proba]
    # Calculate Matthews Correlation Coefficient
    mcc = matthews_corrcoef(Train_Class, predicted_train)
    
    return {'loss': 1-mcc, 'status': STATUS_OK, 'model': model, 'MCC': mcc}

# Search for optimal parameters:
trials = Trials()
best_dnn_classification = fmin(hyperparameter_tuning_DNN, space, algo = tpe.suggest, 
                               max_evals = 10, trials = trials)
best_params = space_eval(space, best_dnn_classification)

# Optimal parameters:
best_params
# Save AUC and MCC of best model to a CSV
results = pd.DataFrame({'Best Parameters': [best_params], 'MCC': [trials.best_trial['result']['MCC']]})
results.to_csv("best_model_results.csv", index=False)
# Train the DNN model on the training molecules using optimal parameters:
tf.random.set_seed(0)
dnn_grid = keras.Sequential([
    layers.Dense(units = int(best_params['units1']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dropout(best_params['dropout1']),
    layers.Dense(units = int(best_params['units2']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.BatchNormalization(),
    layers.Dense(units = int(best_params['units3']), kernel_regularizer = regularizers.l2(0.001), activation = "relu"),
    layers.Dropout(best_params['dropout2']),
    layers.Dense(1, activation = "sigmoid")    
])
dnn_grid.compile(optimizer = best_params['optimizer'], loss = "binary_crossentropy", metrics = ['accuracy'])
dnn_grid.fit(np.array(train_grid_features), y_train, epochs = 100, 
             batch_size = int(best_params['batch_size']), verbose = False)

# Test the DNN model on the test molecules:
prediction_test_dnn_grid_prob = dnn_grid.predict(np.array(test_grid_features))
prediction_test_dnn_grid_class = ['Active' if num >= 0.5 else "Inactive" for num in prediction_test_dnn_grid_prob]

# Get virtual screening results on the test molecules and export results to a CSV file:
grid_result_dnn = pd.DataFrame({"Active_Prob": prediction_test_dnn_grid_prob[:, 0],
                                "Predicted_Class": prediction_test_dnn_grid_class,
                                "Real_Class": Test_Class})
grid_result_dnn.to_csv("Provide_the_pathway_to_your_file_where_save_the_result_run.csv")
